In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

## Backpropagation
* Basado en regla de la cadena
* Base de algoritmos como la gradiente descendente
* Facil de hacer en pytorch
* Cada propagacion se calcula funcion de perdida, la cual se quiere MINIMIZAR
* Tres pasos:
    1- Forward pass: Computar perdida
    2- Computar gradientes locales
    3- Backward pass: Computar dPerdida/dWeights usando regla de la cadena
    

In [2]:
x = torch.tensor(1.0) #Entrada x
y = torch.tensor(2.0) #Entrada y

w = torch.tensor(1.0, requires_grad=True) #Peso inicial
# fordward pass amd compute the loss
y_hat = w * x #
loss = (y_hat - y)**2 # Funcion de perdida
print(loss)

# backward pass
### pytorch calcula los gradientes locales, solo hay que hacer esto
loss.backward()
print(w.grad)

#### update pesos
### next foward and backward etc...


tensor(1., grad_fn=<PowBackward0>)
tensor(-2.)


## Gradiente Descendente: Manual vs Pytorch
Se compararan distintas formas de aplicar el algorimo



### FULL MANUAL
* Prediccion: Manual
* Calcular Gradiente: Manual
* Calcular perdida o loss: Manual
* Actualizar Parametris: Manual
Solo usand numpy

In [3]:
# Prediccion de f(x) = 2 * x
X = np.array([1,2,3,4], dtype=np.float32) #Valores de entrada
Y = np.array([2,4,6,8], dtype=np.float32) #Valores que intento predecir

w = 0.0
# model prediction
def forward(x):
    return w*x

# loss = MSE
def loss(y,y_predicted):
    return((y_predicted-y)**2).mean()

# gradient
# MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2x*(w*x-y)
def gradient(x,y,y_predicted):
    return np.dot(2*x,y_predicted-y).mean()
print(f'Prediction before training: f(5) = {forward(5):.3f}')

#Training
learning_rate=0.01
n_iters=5

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)
    # loss
    l = loss(Y,y_pred)
    # gradients
    dw = gradient(X,Y,y_pred)
    # update weights 
    w -= learning_rate * dw
    
    if epoch % 1 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss ={l:.8f}')
        
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 1.200, loss =30.00000000
epoch 2: w = 1.680, loss =4.79999924
epoch 3: w = 1.872, loss =0.76800019
epoch 4: w = 1.949, loss =0.12288000
epoch 5: w = 1.980, loss =0.01966083
Prediction after training: f(5) = 9.898


Cada paso de entrenamiento aumenta el peso y disminuye la perdida a casi a 0
La prediccion despues del entrenamiento es casi 10 (valor correcto)
Si aumento las epocas o cambio el learning_rate podria acercarme mas rapido, pero tambien PODRIA CONVERGER (hay que tener ojo)

### Semi manual usando torch
* Prediccion: Manual
* Calcular Gradiente: **Pytorch con AUTOGRAD**
* Calcular perdida o loss: Manual
* Actualizar Parametris: Manual

In [4]:
# Prediccion de f(x) = 2 * x
X = torch.tensor([1,2,3,4], dtype=torch.float32) #Valores de entrada
Y = torch.tensor([2,4,6,8], dtype=torch.float32) #Valores que intento predecir

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True) #TODO DEBE SER TENSOR Y DEL MISMO TIPO!! 
# Y RECORDAR EL REQUIRES GRAD

# model prediction
def forward(x):
    return w*x

# loss = MSE
def loss(y,y_predicted):
    return((y_predicted-y)**2).mean()

# gradient
# MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2x*(w*x-y)
# Ya no es necesario la fnucion de gradiente gradiente


print(f'Prediction before training: f(5) = {forward(5):.3f}')

#Training
learning_rate=0.01
n_iters=100

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)
    # loss
    l = loss(Y,y_pred)
    # gradients = backward pass
    l.backward() #dl/dw
    
    # update weights 
    # Recordar curso 2, NO QUEREMOS QUE SE ACOMULEN LOS GRAD
    with torch.no_grad():
        w -= learning_rate * w.grad
    
    # zero gradients: hay que asegurarse que los gradientes SEAN CEROS nuevamente
    w.grad.zero_()
    
    if epoch % 10 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss ={l:.8f}')
        
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss =30.00000000
epoch 11: w = 1.665, loss =1.16278565
epoch 21: w = 1.934, loss =0.04506890
epoch 31: w = 1.987, loss =0.00174685
epoch 41: w = 1.997, loss =0.00006770
epoch 51: w = 1.999, loss =0.00000262
epoch 61: w = 2.000, loss =0.00000010
epoch 71: w = 2.000, loss =0.00000000
epoch 81: w = 2.000, loss =0.00000000
epoch 91: w = 2.000, loss =0.00000000
Prediction after training: f(5) = 10.000


Funciona pero no del todo correcto, eso es por que el backpropagation no es tan exacta como el calculo manual. Ahora si aumentamos las iteraciones llegamos al resultado.

# Training Pipeline
### UN POCO MAS AUTOMATIZADO
* Prediccion: Manual
* Calcular Gradiente: **Pytorch con AUTOGRAD**
* Calcular perdida o loss: **Pytorch Loss**
* Actualizar Parametris: **Pytorch Optimizer**


In [5]:
## Recordar los pasos
# 1) Diseñar un modelo: (inputs, output size, forward pass (operaciones, capas))
# 2) Construir loss y optimizer
# 3) Training loop
# - forward pass: computar prediccion
# - backward pass: gradients
# - update weights

#IMPORTANTE DESDE AHORA IMPORTAR ESTO
# import torch.nn as nn
## Da acceso a varias funciones para automatizar mas weas con la libreria

# Prediccion de f(x) = 2 * x
X = torch.tensor([1,2,3,4], dtype=torch.float32) #Valores de entrada
Y = torch.tensor([2,4,6,8], dtype=torch.float32) #Valores que intento predecir

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True) #TODO DEBE SER TENSOR Y DEL MISMO TIPO!! 
# Y RECORDAR EL REQUIRES GRAD

#Training parametros
learning_rate=0.01
n_iters=100

# model prediction
def forward(x):
    return w*x

# loss = MSE
# Ya no es necesario una funcion manual de Loss
loss = nn.MSELoss()
# Hay qie definir un optimizador, en este caso el SGD
optimizer = torch.optim.SGD([w], lr=learning_rate)


# gradient
# MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2x*(w*x-y)
# Ya no es necesario la fnucion de gradiente gradiente


print(f'Prediction before training: f(5) = {forward(5):.3f}')



for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)
    # loss
    l = loss(Y,y_pred)
    # gradients = backward pass
    l.backward() #dl/dw
    
    # update weights 
    # Recordar curso 2, NO QUEREMOS QUE SE ACOMULEN LOS GRAD
    # Ya no hay que actualizarlos manualmente, basta con el optimizer
    optimizer.step()
    
    # zero gradients: hay que asegurarse que los gradientes SEAN CEROS nuevamente
    ## En este caso se actualizan las gradientes con el optimizador
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss ={l:.8f}')
        
print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss =30.00000000
epoch 11: w = 1.665, loss =1.16278565
epoch 21: w = 1.934, loss =0.04506890
epoch 31: w = 1.987, loss =0.00174685
epoch 41: w = 1.997, loss =0.00006770
epoch 51: w = 1.999, loss =0.00000262
epoch 61: w = 2.000, loss =0.00000010
epoch 71: w = 2.000, loss =0.00000000
epoch 81: w = 2.000, loss =0.00000000
epoch 91: w = 2.000, loss =0.00000000
Prediction after training: f(5) = 10.000


### FULL AUTOMATIZADO
* Prediccion: **Pytorch MODEL**
* Calcular Gradiente: **Pytorch con AUTOGRAD**
* Calcular perdida o loss: **Pytorch Loss**
* Actualizar Parametris: **Pytorch Optimizer**

In [10]:
### Lo mismo con Foward con pytorch
# 1) Diseñar un modelo: (inputs, output size, forward pass (operaciones, capas))
# 2) Construir loss y optimizer
# 3) Training loop
# - forward pass: computar prediccion
# - backward pass: gradients
# - update weights

#Training parametros
learning_rate=0.01
n_iters=1000

#IMPORTANTE DESDE AHORA IMPORTAR ESTO
# import torch.nn as nn
## Da acceso a varias funciones para automatizar mas weas con la libreria

# Prediccion de f(x) = 2 * x
#X = torch.tensor([1,2,3,4], dtype=torch.float32) #Valores de entrada
#Y = torch.tensor([2,4,6,8], dtype=torch.float32) #Valores que intento predecir
## Hay que cambiar la forma de representar el tensor, en vez de un vector fila que sea un vector columna
X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32) #Valores de entrada
Y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32) #Valores que intento predecir
#¿porque? para poder separar entre muestras y cara cteristicas de cada muestra y xq el modelo
## lee vectores verticales como se representan en la mate real
n_samples, n_features = X.shape #OJO QUE ES EL VALOR DE TAMAÑO, NO LOS VALORES COMO TA
print(n_samples,n_features)
# Tambien debemos definir nuestro conjunto TEST, esto es VALORES QUE NO SABEMOS E INTENTAMOS PREDEICR SIN SABER CUAL ES EL RESTULADO
X_test = torch.tensor([5], dtype=torch.float32) 

input_size = n_features
output_size = n_features



#EN ESTE CASO, NO SE NECESITA NISIQUIERA ESPECIFICAR LOS WEIGHTS MANUALMENTE COMO UN TENSOR
#w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True) #TODO DEBE SER TENSOR Y DEL MISMO TIPO!! 
# Y RECORDAR EL REQUIRES GRAD
# model prediction
#def forward(x):
#    return w*x
## Vamos a crear una regresion lineal con 1 capa
model = nn.Linear(input_size, output_size)

# Para obtener la prediccion before training ahora basta con hacer
print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

# loss = MSE
# Ya no es necesario una funcion manual de Loss
loss = nn.MSELoss()
# Hay qie definir un optimizador, en este caso el SGD
#optimizer = torch.optim.SGD([w], lr=learning_rate)
#EN VEZ DE w USAMOS model.parameters(). Hay varios tipos de optimizadors, como SGD o Adam
#SGD
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#ADAM
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# gradient
# MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2x*(w*x-y)
# Ya no es necesario la fnucion de gradiente gradiente






for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = model(X)
    # loss
    l = loss(Y,y_pred)
    # gradients = backward pass
    l.backward() #dl/dw
    
    # update weights 
    # Recordar curso 2, NO QUEREMOS QUE SE ACOMULEN LOS GRAD
    # Ya no hay que actualizarlos manualmente, basta con el optimizer
    optimizer.step()
    
    # zero gradients: hay que asegurarse que los gradientes SEAN CEROS nuevamente
    ## En este caso se actualizan las gradientes con el optimizador
    optimizer.zero_grad()
    
    if epoch % 1000 == 0:
        # Puedo desempaquetar los parametros del modelo para poder ver como van, pero ojo que son lista de listas
        [w, b] = model.parameters()
        print(f'epoch {epoch+1}: w = {w[0][0]:.3f}, loss ={l:.8f}')
        
print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

4 1
Prediction before training: f(5) = 3.271
epoch 1: w = 0.973, loss =15.10378647
Prediction after training: f(5) = 10.003


El resultado es bueno pero no tanto como antes, esto puede ser por que la iniciaclizacion ahora es aleatoria, y la tecnica de optimizacion podria no ser la mejor. Quizas si juego con el learning_rate y iteraciones consiga mejor resultado. De todas formas funciona practicamente perfecto.
Observar que si ocupo optimizador Adam y SGD tambien se llegan a los mismos resultados pero Adam converge mucho mas lento con los parametros de SGD.

En los caos anterior teniamos solo una capa, supongamos que tenemos varias. En ese caso es mejor representar todo como una clase:


In [15]:
### Lo mismo con Foward con pytorch
# 1) Diseñar un modelo: (inputs, output size, forward pass (operaciones, capas))
# 2) Construir loss y optimizer
# 3) Training loop
# - forward pass: computar prediccion
# - backward pass: gradients
# - update weights



#IMPORTANTE DESDE AHORA IMPORTAR ESTO
# import torch.nn as nn
## Da acceso a varias funciones para automatizar mas weas con la libreria

# Prediccion de f(x) = 2 * x
#X = torch.tensor([1,2,3,4], dtype=torch.float32) #Valores de entrada
#Y = torch.tensor([2,4,6,8], dtype=torch.float32) #Valores que intento predecir
## Hay que cambiar la forma de representar el tensor, en vez de un vector fila que sea un vector columna
X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32) #Valores de entrada
Y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32) #Valores que intento predecir
#¿porque? para poder separar entre muestras y cara cteristicas de cada muestra y xq el modelo
## lee vectores verticales como se representan en la mate real
n_samples, n_features = X.shape #OJO QUE ES EL VALOR DE TAMAÑO, NO LOS VALORES COMO TA
print(n_samples,n_features)
# Tambien debemos definir nuestro conjunto TEST, esto es VALORES QUE NO SABEMOS E INTENTAMOS PREDEICR SIN SABER CUAL ES EL RESTULADO
X_test = torch.tensor([5], dtype=torch.float32) 

input_size = n_features
output_size = n_features

## Con el formato clase puedo definir mas capas con mayor facilidad
class LinearRegression(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LinearRegression,self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.lin(x)
        

## Vamos a crear una regresion lineal con 1 capa
#model = nn.Linear(input_size, output_size)
model = LinearRegression(input_size,output_size)

# Para obtener la prediccion before training ahora basta con hacer
print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

#Training parametros
learning_rate=0.01
n_iters=1000


# loss = MSE
# Ya no es necesario una funcion manual de Loss
loss = nn.MSELoss()
# Hay qie definir un optimizador, en este caso el SGD
#optimizer = torch.optim.SGD([w], lr=learning_rate)
#EN VEZ DE w USAMOS model.parameters(). Hay varios tipos de optimizadors, como SGD o Adam
#SGD
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#ADAM
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = model(X)
    # loss
    l = loss(Y,y_pred)
    # gradients = backward pass
    l.backward() #dl/dw
    
    # update weights 
    # Recordar curso 2, NO QUEREMOS QUE SE ACOMULEN LOS GRAD
    # Ya no hay que actualizarlos manualmente, basta con el optimizer
    optimizer.step()
    
    # zero gradients: hay que asegurarse que los gradientes SEAN CEROS nuevamente
    ## En este caso se actualizan las gradientes con el optimizador
    optimizer.zero_grad()
    
    if epoch % 100 == 0:
        # Puedo desempaquetar los parametros del modelo para poder ver como van, pero ojo que son lista de listas
        [w, b] = model.parameters()
        print(f'epoch {epoch+1}: w = {w[0][0]:.3f}, loss ={l:.8f}')
        
print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

4 1
Prediction before training: f(5) = 2.910
epoch 1: w = 0.677, loss =13.65275669
epoch 101: w = 1.763, loss =0.08163604
epoch 201: w = 1.824, loss =0.04481722
epoch 301: w = 1.870, loss =0.02460414
epoch 401: w = 1.904, loss =0.01350738
epoch 501: w = 1.929, loss =0.00741537
epoch 601: w = 1.947, loss =0.00407095
epoch 701: w = 1.961, loss =0.00223491
epoch 801: w = 1.971, loss =0.00122694
epoch 901: w = 1.978, loss =0.00067358
Prediction after training: f(5) = 9.967
